In [ ]:
# Importações necessárias
from utils.Agente_oo import AgenteCharla
from utils.pdf_to_img_to_text import extrair_texto_de_pdf_scaneado

from pathlib import Path


In [ ]:
agent = AgenteCharla()

In [ ]:
# Verificação da pasta de notas fiscais
pasta_notas = Path('nota_fiscal')
if not pasta_notas.exists():
    print(f"Pasta '{pasta_notas}' não encontrada.")

# Busca todos os arquivos PDF na pasta
arquivos_pdf = list(pasta_notas.glob('*.pdf'))
if not arquivos_pdf:
    print("Nenhum arquivo PDF encontrado na pasta 'nota_fiscal'.")

print(f"Encontrados {len(arquivos_pdf)} arquivos PDF para processar...")

In [ ]:
for i, arquivo_pdf in enumerate(arquivos_pdf, 1):
    arquivo_txt = extrair_texto_de_pdf_scaneado(arquivo_pdf)

In [ ]:
arquivo_txt

In [ ]:
res_txt = await agent.extrair_dados_txt(arquivo_txt)

In [ ]:
res_txt.model_dump()

In [ ]:
pdf_nota = 'nota_fiscal/NFSe3.pdf'
arquivobyte = Path(pdf_nota).read_bytes()



In [ ]:
resultado = await agent.extrair_dados_img(arquivobyte)

In [ ]:
resultado.model_dump()

In [ ]:
agent.agent